In [483]:
import os 
import time 
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import io
from io import BytesIO
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import pathlib
import cv2

In [568]:
DRIVER_PATH = '/Users/trey/Image-Scraping/scraping/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
search_url = "https://www.google.com/search?q={q}&sxsrf=ALeKk02zAb9RaNNb-qSenTEJh1i2XX480w:1613489053802&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjChJqP2-7uAhVyTTABHdX0CPoQ_AUoAXoECAcQAw&biw=767&bih=841"
wd.get(search_url.format(q='dog'))
body = wd.find_element_by_tag_name("body")
    
for i in range(75):
    time.sleep(.5)
    try:    
        body.send_keys(Keys.PAGE_DOWN)
    except Exception:
        pass
    
click = wd.find_element_by_class_name('mye4qd')
click.click()

for i in range(75):
    time.sleep(.5)
    try:
        body.send_keys(Keys.PAGE_DOWN)
    except Exception:
        pass 
# retry  = class = 'sGx53d'    
data = wd.find_elements_by_class_name("rg_i.Q4LuWd")
links1 = [data[i].get_attribute('src') for i in range(len(data))]
links2 = [data[i].get_attribute('data-src') for i in range(len(data))]
test = links1 + links2
new = []
for i in test:
    try:
        if type(i) != None:
            if i[0:4] == 'data':
                i = i.replace("data:image/jpeg;base64,","")
                if i[-2:] != '==':
                    new_edit = new + '=='
                    i = (Image.open(BytesIO(base64.b64decode(i)))).resize((150,150))
                    new.append(i)
                else:
                    i = (Image.open(BytesIO(base64.b64decode(i)))).resize((150,150))
                    new.append(i)
            if i[0:4] == 'http':
                i = requests.get(i)
                i = Image.open(BytesIO(i.content))
                new.append(i)
        if type(i) == None:
            pass
    except Exception:
        pass


In [570]:
len(new)

884

In [575]:
def scrape(search_name,web_driver=wd):
    
    DRIVER_PATH = '/Users/trey/Image-Scraping/scraping/chromedriver'
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    search_url = "https://www.google.com/search?q={q}&sxsrf=ALeKk02zAb9RaNNb-qSenTEJh1i2XX480w:1613489053802&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjChJqP2-7uAhVyTTABHdX0CPoQ_AUoAXoECAcQAw&biw=767&bih=841"
    wd.get(search_url.format(q=search_name))
    body = wd.find_element_by_tag_name("body")
    
    for i in range(75):
        time.sleep(.5)
        try:    
            body.send_keys(Keys.PAGE_DOWN)
        except Exception:
            pass
    
    click = wd.find_element_by_class_name('mye4qd')
    click.click()

    for i in range(75):
        time.sleep(.5)
        try:
            body.send_keys(Keys.PAGE_DOWN)
        except Exception:
            pass 
        
    data = wd.find_elements_by_class_name("rg_i.Q4LuWd")
    links1 = [data[i].get_attribute('src') for i in range(len(data))]
    links2 = [data[i].get_attribute('data-src') for i in range(len(data))]
    test = links1 + links2

    new = []
    for i in test:
        try:
            if type(i) != None:
                if i[0:4] == 'data':
                    i = i.replace("data:image/jpeg;base64,","")
                    if i[-2:] != '==':
                        new_edit = new + '=='
                        i = (Image.open(BytesIO(base64.b64decode(i)))).resize((150,150))
                        new.append(i)
                    else:
                        i = (Image.open(BytesIO(base64.b64decode(i)))).resize((150,150))
                        new.append(i)
                if i[0:4] == 'http':
                    i = requests.get(i)
                    i = Image.open(BytesIO(i.content))
                    new.append(i)
            if type(i) == None:
                pass
        except Exception:
            pass
    return new

    
def save(list_of_images,class_of_image,valid_ratio):

    valid = float(valid_ratio * float(len(list_of_images)))
    valid_data = []
    training_data = [] 
    int_valid = int(valid)
    
    for i in range(len(list_of_images)):
        if i < int_valid:
            valid_data.append(list_of_images[i])
        else:
            training_data.append(list_of_images[i])
    
    class_type = str(class_of_image)
    test_path = 'Images/training/' + class_type
    valid_path = 'Images/validation/' + class_type
    
    try:
        os.mkdir('Images')
        os.mkdir('Images/training')
        os.mkdir('Images/validation')
    except Exception:
        pass
    
    try:
        os.mkdir(test_path)
        os.mkdir(valid_path)
    except Exception:
        pass
    
    index = 0
    for i in list_of_images:
        try:
            i.save(file_path + '/' + class_type + str(index) + '.jpeg')
            index +=1
        except Exception:
            pass


    

    
   


In [576]:
save(dog,'dog',.2)

In [573]:
dog = scrape('dog')

In [574]:
len(dog)

1425

In [538]:
def augment(new):    
    class_type = str(new)
    file_path = 'Images/training/' + class_type 
    
    try:
        os.mkdir(file_path)
    except Exception:
        pass
    
    for i in range(len(new)):
        index = 0 
        image = np.expand_dims(new[i],axis=0)
    
        for batch in data_gen.flow(image, 
                           batch_size=32,
                          save_to_dir=file_path, 
                           save_prefix= class_type, 
                           save_format='jpeg'):
            index += 1 
            if index > 20:
                break

In [517]:
#dog = scrape('dog')
cat = scrape('cat')

In [518]:
dog = scrape('dog')

In [539]:
augment(dog)

OSError: [Errno 63] File name too long: 'Images/training/[<PIL.Image.Image image mode=RGB size=150x150 at 0x140B93EB8>, <PIL.Image.Image image mode=RGB size=150x150 at 0x140B928D0>, <PIL.Image.Image image mode=RGB size=150x150 at 0x1385A1198>, <PIL.Image.Image image mode=RGB size=150x150 at 0x1385A1F28>, <PIL.Image.Image image mode=RGB size=150x150 at 0x1385A1CC0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x168 at 0x1385A1BE0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=299x168 at 0x1385A10F0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=275x183 at 0x1385A1240>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=324x156 at 0x1385A1E48>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=275x183 at 0x1385A1208>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=275x183 at 0x1385A1EB8>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=349x144 at 0x1385A1390>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=264x191 at 0x1385

In [524]:
augment(cat)

In [530]:
file = save(cat,'cat')

In [535]:
save(dog,'dog')

In [543]:
testing = 'Images/training'
train_datagen = ImageDataGenerator()

In [544]:

train_generator = train_datagen.flow_from_directory(
        testing,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical') 

#validation_generator = test_datagen.flow_from_directory(
#        'data/validation',
#        target_size=(150, 150),
#        batch_size=batch_size,
#        class_mode='binary')

Found 74 images belonging to 2 classes.


In [548]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3),padding='same', input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Conv2D(32, (3, 3),padding='same',))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same',))

model.add(Conv2D(64, (3, 3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#model.save_weights('first_try.h5')

In [550]:
batch_size = 32
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

NameError: name 'validation_generator' is not defined